In [ ]:
#@title Imports & Set Path


import pandas_profiling
import pandas as pd
from random import randint
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Dense, LSTM
from keras.layers import Add, Average 
from keras.layers import Dropout, Activation, Input, Flatten
from keras.layers import TimeDistributed, Bidirectional
from keras import optimizers
from keras import regularizers
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split





path = 'Data/' #add the path of your data

In [ ]:
#@title reading from google colab drive 
from google.colab import drive 
drive.mount('/content/gdrive')

path = '/content/gdrive/My Drive/ChallengeUP-master/ChallengeUP-master/Data/'

base = pd.read_csv(path + 'CompleteDataSet_training_competition_fixed.csv')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
base.head()

,Unnamed: 0,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__cwt_coefficie

In [ ]:
base.describe()

,Unnamed: 0,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__cwt_coeff

In [ ]:
base.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 120 columns):
Unnamed: 0                                                                                       int64
AnkleAccelerometer x-axis__abs_energy                                                            float64
AnkleAccelerometer x-axis__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"          float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__quantile__q_0.1          

In [ ]:
base.shape

(31962, 120)

In [ ]:
base.isnull().sum()

Unnamed: 0                                                                                 0
AnkleAccelerometer x-axis__abs_energy                                                      0
AnkleAccelerometer x-axis__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"    0
AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)           0
AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)           0
                                                                                          ..
WristAccelerometer x-axis__quantile__q_0.2                                                 0
WristAccelerometerMag__quantile__q_0.8                                                     0
WristLuminosity illuminance__abs_energy                                                    0
TimeStamps                                                                                 0
Tag                                                                   

In [ ]:
base['TimeStamps'] = pd.to_datetime(base['TimeStamps'])


In [ ]:
abs(base.corr()["Tag"].sort_values(ascending=False))

Tag                                                                                       1.000000
NeckAccelerometer z-axis__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"    0.522484
NeckAccelerometer x-axis__benford_correlation                                             0.504052
RightPocketAccelerometer z-axis__abs_energy                                               0.407972
BeltAccelerometer y-axis__benford_correlation                                             0.394901
                                                                                            ...   
NeckAccelerometerMag__c3__lag_3                                                           0.264218
RightPocketAccelerometer x-axis__abs_energy                                               0.267628
NeckAccelerometerMag__benford_correlation                                                 0.392680
BeltAccelerometer y-axis__fft_coefficient__attr_"abs"__coeff_0                            0.430653
AnkleAccel

In [ ]:
# base.profile_report()

In [ ]:
Falling1 = (base["Tag"].values==1).sum()
Falling2 = (base["Tag"].values==2).sum()
Falling3 = (base["Tag"].values==3).sum()
Falling4 = (base["Tag"].values==4).sum()
Falling5 = (base["Tag"].values==5).sum()
Walking = (base["Tag"].values==6).sum()
Standing = (base["Tag"].values==7).sum()
Sitting = (base["Tag"].values==8).sum()
Object = (base["Tag"].values==9).sum()
Jumping = (base["Tag"].values==10).sum()
Laying = (base["Tag"].values==11).sum()
Other = (base["Tag"].values==20).sum()

In [ ]:
frequency_dict = {"Fal1": Falling1,
                  "Fal2": Falling2,
                  "Fal3": Falling3,
                  "Fal4": Falling4,
                  "Fal5": Falling5,
                  "Walk": Walking,
                  "Stand": Standing,
                  "Sit": Sitting,
                  "Pick": Object,
                  "Jump": Jumping,
                  "Lay": Laying,
                  "Other" : Other}

In [ ]:
frequency = pd.DataFrame(list(frequency_dict.items()),columns=["Activity","Count"])
frequency.head(12)

,Activity,Count
0,Fal1,155
1,Fal2,143
2,Fal3,195
3,Fal4,170
4,Fal5,200
5,Walk,5911
6,Stand,6826
7,Sit,5766
8,Pick,198
9,Jump,2873


In [ ]:
y = base["Tag"]
y = pd.DataFrame(y,columns=["Tag"])


In [ ]:
x = base.drop(["Tag","TimeStamps","Unnamed: 0"],axis=1)

In [ ]:
x.head()

,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",BeltAccelerometerMag__median,...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccel

In [ ]:
x.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 117 columns):
AnkleAccelerometer x-axis__abs_energy                                                            float64
AnkleAccelerometer x-axis__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"          float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)                 float64
AnkleAccelerometer x-axis__quantile__q_0.1                                                       float64
AnkleAccelerometer y-axis__maximum                

In [ ]:
y.head()

,Tag
0,7
1,7
2,7
3,7
4,7


In [ ]:
x.shape

(31962, 117)

In [ ]:
y.shape

(31962, 1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
def normalize_(x):
  scaler=StandardScaler()
  return scaler.fit_transform(x)

In [ ]:
x.shape

(31962, 117)

In [ ]:
y.shape

(31962, 1)

In [ ]:
x.head()

,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",BeltAccelerometerMag__median,...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccel

In [ ]:
y.head()

,Tag
0,7
1,7
2,7
3,7
4,7


In [ ]:


# Random Forest

clf = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    warm_start=False,
    class_weight='balanced'
)



algos_name=['RandomForest']
accuracy = []
precision = []
recall = []
f1 = []
result=pd.DataFrame(columns=['AccuracyScore','PrecisionScore','RecallScore','f1_Score'],index=algos_name)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
clf.fit(x_train,y_train)

predict = clf.predict(x_test)

accuracy.append(accuracy_score(y_test,predict))
precision.append(precision_score(y_test,predict,average='weighted'))
recall.append(recall_score(y_test,predict,average='weighted'))
f1.append(f1_score(y_test,predict,average='weighted'))

result.AccuracyScore=accuracy
result.PrecisionScore=precision
result.RecallScore=recall
result.f1_Score=f1

result.sort_values('f1_Score',ascending=False)

,AccuracyScore,PrecisionScore,RecallScore,f1_Score
RandomForest,0.928294,0.944566,0.928294,0.934587


In [ ]:
importances = clf.feature_importances_

for i,v in enumerate(importances):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.08286
Feature: 1, Score: 0.00761
Feature: 2, Score: 0.00198
Feature: 3, Score: 0.00425
Feature: 4, Score: 0.00280
Feature: 5, Score: 0.00328
Feature: 6, Score: 0.00796
Feature: 7, Score: 0.02019
Feature: 8, Score: 0.01524
Feature: 9, Score: 0.02304
Feature: 10, Score: 0.00655
Feature: 11, Score: 0.00599
Feature: 12, Score: 0.00167
Feature: 13, Score: 0.00160
Feature: 14, Score: 0.00351
Feature: 15, Score: 0.00526
Feature: 16, Score: 0.00670
Feature: 17, Score: 0.00460
Feature: 18, Score: 0.00298
Feature: 19, Score: 0.02405
Feature: 20, Score: 0.00214
Feature: 21, Score: 0.00076
Feature: 22, Score: 0.00201
Feature: 23, Score: 0.00165
Feature: 24, Score: 0.00033
Feature: 25, Score: 0.00059
Feature: 26, Score: 0.00680
Feature: 27, Score: 0.01278
Feature: 28, Score: 0.02078
Feature: 29, Score: 0.00778
Feature: 30, Score: 0.00351
Feature: 31, Score: 0.00733
Feature: 32, Score: 0.01749
Feature: 33, Score: 0.00682
Feature: 34, Score: 0.00127
Feature: 35, Score: 0.01487
Fe

In [ ]:
path = '/content/gdrive/My Drive/ChallengeUP-master/ChallengeUP-master/Data/'

test = pd.read_csv(path + 'CompleteDataSet_testing_competition_fixed.csv')

In [ ]:
test.head()

,Unnamed: 0,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__cwt_coeff

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Columns: 119 entries, Unnamed: 0 to TimeStamps
dtypes: float64(117), int64(1), object(1)
memory usage: 9.2+ MB


In [ ]:
test_time = test

In [ ]:
test = test.drop(["TimeStamps"],axis=1)

In [ ]:
test.head()

,Unnamed: 0,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccelerometerMag__cwt_c

In [ ]:
test = test.drop(["Unnamed: 0"],axis=1)

In [ ]:
# predictions = classification_(X_train,Y_train,prediction = test_df)
predictions = clf.predict(test)

In [ ]:
predictions

array([11,  3,  3, ..., 11, 11, 11])

In [ ]:
# submission = pd.DataFrame(predictions, columns=["Tag_1" ,	"Tag_2" ,	"Tag_3" ,	"Tag_4 ",	"Tag_5 ","Tag_6","Tag_7" ,"Tag_8" ,"Tag_9 ","Tag_10","Tag_11" ,"Tag_20"]) #for dummies
submission = pd.DataFrame(predictions, columns=["Target"])
submission.head()

,Target
0,11
1,3
2,3
3,10
4,3


In [ ]:
test_time

,Unnamed: 0,AnkleAccelerometer x-axis__abs_energy,"AnkleAccelerometer x-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","AnkleAccelerometer x-axis__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometer x-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometer x-axis__quantile__q_0.1,AnkleAccelerometer y-axis__maximum,AnkleAccelerometerMag__abs_energy,"AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","AnkleAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","AnkleAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)","AnkleAccelerometerMag__cwt_coefficients__coeff_8__w_2__widths_(2, 5, 10, 20)",AnkleAccelerometerMag__quantile__q_0.3,AnkleAccelerometerMag__quantile__q_0.4,AnkleLuminosity illuminance__abs_energy,BeltAccelerometer x-axis__maximum,BeltAccelerometer y-axis__abs_energy,"BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometer y-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""",BeltAccelerometer y-axis__benford_correlation,BeltAccelerometer y-axis__c3__lag_1,BeltAccelerometer y-axis__c3__lag_2,"BeltAccelerometer y-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer y-axis__fft_coefficient__attr_""abs""__coeff_0","BeltAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""",BeltAccelerometer z-axis__c3__lag_1,"BeltAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","BeltAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)",BeltAccelerometerMag__abs_energy,"BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""","BeltAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","BeltAccelerometerMag__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)","BeltAccelerometerMag__fft_aggregated__aggtype_""centroid""",...,"RightPocketAccelerometer x-axis__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer x-axis__cwt_coefficients__coeff_7__w_2__widths_(2, 5, 10, 20)",RightPocketAccelerometer x-axis__minimum,RightPocketAccelerometer x-axis__variation_coefficient,RightPocketAccelerometer y-axis__maximum,RightPocketAccelerometer y-axis__minimum,RightPocketAccelerometer z-axis__abs_energy,"RightPocketAccelerometer z-axis__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_6__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometer z-axis__cwt_coefficients__coeff_9__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",RightPocketAccelerometerMag__c3__lag_1,RightPocketAccelerometerMag__c3__lag_2,RightPocketAccelerometerMag__c3__lag_3,"RightPocketAccelerometerMag__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)","RightPocketAccelerometerMag__cwt_coeff

In [ ]:
test_time = test_time["TimeStamps"]


In [ ]:
test_time

0        2018-07-11 17:48:46.790234
1        2018-07-11 17:48:47.070993
2        2018-07-11 17:48:47.339718
3        2018-07-11 17:48:47.647542
4        2018-07-11 17:48:47.897709
                    ...            
10124    2018-07-12 12:18:28.194109
10125    2018-07-12 12:18:28.442270
10126    2018-07-12 12:18:28.692437
10127    2018-07-12 12:18:29.025324
10128    2018-07-12 12:18:29.070446
Name: TimeStamps, Length: 10129, dtype: object

In [ ]:
# base['TimeStamps'] = pd.to_datetime(base['TimeStamps'])

timestamps = pd.DataFrame(test_time, columns=["TimeStamps"])
timestamps["TimeStamps"] = pd.to_datetime(timestamps["TimeStamps"])
# timestamps = timestamps.drop(timestamps.index[0])
timestamps = timestamps.reset_index()
timestamps = timestamps.drop(["index"],axis=1)
timestamps
timestamps2 = timestamps['TimeStamps'].apply(
      lambda x: x.replace(microsecond=0))


In [ ]:
timestamps2 = pd.DataFrame(timestamps2,columns=["TimeStamps"])
timestamps2

,TimeStamps
0,2018-07-11 17:48:46
1,2018-07-11 17:48:47
2,2018-07-11 17:48:47
3,2018-07-11 17:48:47
4,2018-07-11 17:48:47
...,...
10124,2018-07-12 12:18:28
10125,2018-07-12 12:18:28
10126,2018-07-12 12:18:28
10127,2018-07-12 12:18:29


In [ ]:
submission.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10129 entries, 0 to 10128
Data columns (total 1 columns):
Target    10129 non-null int64
dtypes: int64(1)
memory usage: 79.3 KB


In [ ]:
submission_combined = timestamps2.join(submission.astype(int))
submission_combined

,TimeStamps,Target
0,2018-07-11 17:48:46,11
1,2018-07-11 17:48:47,3
2,2018-07-11 17:48:47,3
3,2018-07-11 17:48:47,10
4,2018-07-11 17:48:47,3
...,...,...
10124,2018-07-12 12:18:28,11
10125,2018-07-12 12:18:28,11
10126,2018-07-12 12:18:28,11
10127,2018-07-12 12:18:29,11


In [ ]:
submission_combined.head(20)

,TimeStamps,Target
0,2018-07-11 17:48:46,11
1,2018-07-11 17:48:47,3
2,2018-07-11 17:48:47,3
3,2018-07-11 17:48:47,10
4,2018-07-11 17:48:47,3
5,2018-07-11 17:48:48,4
6,2018-07-11 17:48:48,3
7,2018-07-11 17:48:48,3
8,2018-07-11 17:48:48,11
9,2018-07-11 17:48:49,11


In [ ]:
# test4 = submission_combined.loc[submission_combined["TimeStamps"] == test3 ].mode()
# test4.head()

In [ ]:
submission_combined = submission_combined.groupby(by='TimeStamps', as_index=False)['Target'].agg(pd.Series.mode, )

In [ ]:
submission_combined.head()

,TimeStamps,Target
0,2018-07-11 17:48:46,11
1,2018-07-11 17:48:47,3
2,2018-07-11 17:48:48,3
3,2018-07-11 17:48:49,11
4,2018-07-11 17:48:50,11


In [ ]:
submission_combined['IsArray'] = submission_combined.Target.apply(lambda x: isinstance(x,(list,pd.core.series.Series,np.ndarray)))

In [ ]:
submission_combined = submission_combined.loc[submission_combined.IsArray == False].loc[:, ['TimeStamps','Target']]

In [ ]:
submission_combined.Target = pd.to_numeric(submission_combined.Target, downcast='integer')

In [ ]:
submission_combined.head()

,TimeStamps,Target
0,2018-07-11 17:48:46,11
1,2018-07-11 17:48:47,3
2,2018-07-11 17:48:48,3
3,2018-07-11 17:48:49,11
4,2018-07-11 17:48:50,11


In [ ]:
submission_combined.columns = ["timestamp","target"]

In [ ]:
submission_combined.head()

,timestamp,target
0,2018-07-11 17:48:46,11
1,2018-07-11 17:48:47,3
2,2018-07-11 17:48:48,3
3,2018-07-11 17:48:49,11
4,2018-07-11 17:48:50,11


In [ ]:
# @title Submission File

!pip install -U -q PyDrive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import datetime

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
driveWriter = GoogleDrive(gauth)

folder_id = '1El1JO73c9fdMmqnVkB-8dryE47oyoJXP'
file_title = 'test-results' + "_New3" + '.csv' 

submission_file = driveWriter.CreateFile({
    'title': file_title,
    'parents': [{'kind': 'drive#fileLink', 'id': folder_id}]
})

submission_file.SetContentString(
    submission_combined.to_csv(
        index=False, columns=['timestamp', 'target']
    )
)

submission_file.Upload()